In [1]:
%matplotlib notebook
import sys
from importlib import reload
import pandas as pd
import numpy as np
import csv

from random import randint

reload(sys)

from sklearn.linear_model import LogisticRegression
import xgboost as xgbq
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb


from tqdm import tqdm
tqdm.pandas()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
frame = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Full_Dataset_Games_Standings_2007-2018.csv', index_col=0)

# Remove games played in 2017-2018 season and use that dataset to test instead
import datetime

frame.loc[frame['Result'] == 'H', 'Result'] = 1
frame.loc[frame['Result'] == 'A', 'Result'] = 0

10343


In [3]:
oddsFrame = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Odds 2007-2018 Raw.csv')

def findPayout(date, homeTeam, awayTeam):
    for index, row in oddsFrame.iterrows():
        if date == row['Date'] and row['HomeTeam'] in homeTeam and row['AwayTeam'] in awayTeam:
            return row['HomeOdds'], row['AwayOdds']
    return float('NaN'), float('NaN')

def calculateBetWith100Outcome(prediction, result, homePayout, awayPayout):
    if result == 1 and prediction == 1:
        # Predicted home win correctly
        return homePayout * 100
    elif result == 0 and prediction == 0:
        # Predicted lose correctly
        return awayPayout * 100
    else:
        # Predicted wrong
        return  -100
    
def calculateBetFor100TotalInvestment(prediction, result, homePayout, awayPayout):
    if prediction == 1:
        # Predicted correctly
        return abs(100 / homePayout)
    elif prediction == 0:
        return abs(100 / awayPayout)
    
def calculateBetFor100Outcome(prediction, result, homePayout, awayPayout):
    if result == prediction:
        # Predicted correctly
        return 100
    elif prediction == 0:
        # Predicted lose incorrectly
        return -100 / awayPayout
    elif prediction == 1:
        # Predicted win incorrectly
        return  -100 / homePayout
    
def calculateBetEvenOutcome(prediction, result, homePayout, awayPayout):
    if result == prediction:
    # Predicted home win correctly
        return 100
    else:
    # Predicted wrong
        return  -100
    
def addOdds(row):
    print(row['Date'])
#     import ipdb; ipdb.set_trace()
    homePayout, awayPayout = findPayout(row['Date'], row['HomeTeam'], row['AwayTeam'])
    vegasPrediction = None
    if homePayout < awayPayout:
        vegasPrediction = 1
    elif homePayout > awayPayout:
        vegasPrediction = 0
    else:
        vegasPrediction = randint(0, 1)
    row['VegasPrediction'] = vegasPrediction
    row['HomePayout'] = homePayout
    row['AwayPayout'] = awayPayout
    
#     row['BetEvenOutcome'] = calculateBetEvenOutcome(row['Prediction'], row['Result'], homePayout, awayPayout)
    
#     row['BetWith100Outcome'] = calculateBetWith100Outcome(row['Prediction'], row['Result'], homePayout, awayPayout)
#     row['BetFor100Outcome'] = calculateBetFor100Outcome(row['Prediction'], row['Result'], homePayout, awayPayout)
#     row['BetFor100TotalInvestment'] = calculateBetFor100TotalInvestment(row['Prediction'], row['Result'], homePayout, awayPayout)

    row['VegasBetWith100Outcome'] = calculateBetWith100Outcome(row['VegasPrediction'], row['Result'], homePayout, awayPayout)
    row['VegasBetFor100Outcome'] = calculateBetFor100Outcome(row['VegasPrediction'], row['Result'], homePayout, awayPayout)
    return row
    

frame = frame.progress_apply(addOdds, axis=1)

display(frame)
frame.to_csv('2007-2018 Bet Results.csv')


  0%|          | 0/10343 [00:00<?, ?it/s]

2007/11/02


  0%|          | 2/10343 [00:00<33:26,  5.15it/s]

2007/11/02


  0%|          | 3/10343 [00:00<40:42,  4.23it/s]

...

100%|█████████▉| 10342/10343 [1:20:36<00:00,  1.56it/s]

2018/04/07


100%|██████████| 10343/10343 [1:20:36<00:00,  1.57it/s]

2018/04/07
2018/04/07


,ID,Date,HomeTeam,AwayTeam,CF%,CSh%,CSv%,FF%,FSh%,FSv%,...,Sh%,Sv,FOW%,W%,Result,VegasPrediction,HomePayout,AwayPayout,VegasBetWith100Outcome,VegasBetFor100Outcome
181,https://www.nhl.com/gamecenter/2007020182,2007/11/02,New Jersey Devils,Toronto Maple Leafs,4.42,-1.413540,-0.354732,3.76,-1.988645,-0.079722,...,-3.151893,0.10,1.1,-0.020979,1,1,0.689655,1.350000,68.965517,100.0
182,https://www.nhl.com/gamecenter/2007020183,2007/11/02,Washington Capitals,Philadelphia Flyers,10.24,-2.807782,-1.290578,7.69,-3.223536,-1.473762,...,-4.191705,-1.91,3.3,-0.245455,0,1,0.854701,1.070000,-100.000000,-117.0
183,https://www.nhl.com/gamecenter/2007020184,2007/11/02,Buffalo Sabres,Florida Panthers,-0.13,2.300800,-1.221793,0.94,2.814914,-1.779103,...,3.439909,-2.24,2.2,0.083333,0,1,0.645161,1.450000,-100.000000,-155.0
184,https://www.nhl.com/gamecenter/2007020185,2007/11/02,Dallas Stars,Coyotes,-1.47,1.460398,0.251726,-2.41,1.992685,0.476706,...,3.169516,0.67,-0.5,0.016667,0,1,0.425532,2.150000,-100.000000,-235.0
185,https://www.nhl.com/gamecenter/2007020186,2007/11/02,Edmonton Oilers,Nashville Predators,-2.55,-0.427064,0.131494,-3.35,-0.461974,0.301930,...,-0.533771,-0.01,1.2,0.020979,0,1,0.709220,1.310000,-100.000000,-141.0
186,https://www.nhl.com/gamecenter/2007020187,2007/11/02,San Jose Sharks,Los Angeles Kings,1.60,-0.830282,1.070527,2.45,-0.916607,1.017180,...,-1.096545,1.45,6.1,0.038462,0,1,0.450450,2.020000,-100.000000,-222.0
187,https://www.nhl.com/gamecenter/2007020188,2007/11/03,Montréal Canadiens,Toronto Maple Leafs,1.06,-0.457981,2.341007,-0.36,-0.038425,2.946117,...,-0.406343,3.23,1.1,0.198718,0,1,0.571429,1.650000,-100.000000,-175.0
188,https://www.nhl.com/gamecenter/2007020189,2007/11/03,Ottawa Senators,Boston Bruins,2.66,0.833424,1.068271,3.21,1.333901,1.120448,...,0.807193,2.10,2.9,0.272727,1,1,0.440529,2.070000,44.052863,100.0
189,https://www.nhl.com/gamecenter/2007020190,2007/11/03,New York Islanders,Pittsburgh Penguins,1.47,0.053590,-0.599915,1.65,0.577136,-1.393353,...,-0.028653,-1.66,0.2,0.100000,1,1,0.847458,1.080000,84.745763,100.0
190,https://www.nhl.com/gamecenter/2007020191,2007/11/03,New York Rangers,New Jersey Devils,1.12,-1.887839,3.019908,3.60,-2.535917,3.420050,...,-3.593985,4.48,3.5,0.053030,1,1,0.564972,1.670000,56.497175,100.0


In [19]:
from sklearn.metrics import f1_score

frame = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/2007-2018 Bet Results.csv')
frame = frame.dropna()

# frame = frame.drop(frame[frame['HomePayout'] == frame['AwayPayout']].index)

results = frame[['Result']].values.flatten()
vegasPredictions = frame[['VegasPrediction']].values.flatten()
print("Accuracy: %6.6f%%" % (sum(results == vegasPredictions) / float(len(frame)) * 100))
print("F1 Score: %6.6f%%" % (f1_score(results, vegasPredictions) * 100))

# print("If I bet $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['VegasBetWith100Outcome'].sum(), totalGames * 100))
# vegasPredictions = newGamesFrame[['VegasPrediction']].values.flatten()
# print("Vegas (Favorites)")
# print("     F1 Score: %6.6f" % (f1_score(newy_all, vegasPredictions)))
# print("     Accuracy: %6.6f   (%d correct / %d games)" % (sum(newy_all == vegasPredictions) / float(len(vegasPredictions)), sum(newy_all == vegasPredictions), len(vegasPredictions)))

Accuracy: 58.411169%
F1 Score: 67.461538%


In [4]:
newGamesFrame[['Prediction', 'Result', 'HomePayout', 'AwayPayout', 'BetFor100Outcome', 'BetFor100TotalInvestment']]

NameError: name 'newGamesFrame' is not defined

In [ ]:
print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f (ROI: %6.2f%%)" % (newGamesFrame.shape[0], newGamesFrame['BetFor100Outcome'].sum(), newGamesFrame['BetFor100TotalInvestment'].sum(), (newGamesFrame['BetFor100Outcome'].sum() / newGamesFrame['BetFor100TotalInvestment'].sum() * 100)))
print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['VegasBetFor100Outcome'].sum(), totalGames * 100))
print("If I bet $100 to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['BetEvenOutcome'].sum(), totalGames * 100))

In [ ]:
print('Games where Vegas states there is an equal chance of either team winning')
equalOddsGames = newGamesFrame.loc[newGamesFrame['HomePayout'] == newGamesFrame['AwayPayout']]
display(equalOddsGames)

In [ ]:
totalGames = newGamesFrame.shape[0]
totalGamesWithFavorites = totalGames - equalOddsGames.shape[0]

In [ ]:
print("If I bet $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['BetWith100Outcome'].sum(), totalGames * 100))
print("Classifier")
print("     F1 Score: %6.6f" % (f1_score(newy_all, predictions)))
print("     Accuracy: %6.6f   (%d correct / %d games)" % (sum(newy_all == predictions) / float(len(predictions)), sum(newy_all == predictions), len(predictions)))

In [ ]:
print("If I bet $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['VegasBetWith100Outcome'].sum(), totalGames * 100))
vegasPredictions = newGamesFrame[['VegasPrediction']].values.flatten()
print("Vegas (Favorites)")
print("     F1 Score: %6.6f" % (f1_score(newy_all, vegasPredictions)))
print("     Accuracy: %6.6f   (%d correct / %d games)" % (sum(newy_all == vegasPredictions) / float(len(vegasPredictions)), sum(newy_all == vegasPredictions), len(vegasPredictions)))

In [ ]:
samePrediction = newGamesFrame.loc[newGamesFrame['Prediction'] == newGamesFrame['VegasPrediction']].shape[0]
print("My model is able to identify the favourite team %6.2f%% of the time" % (samePrediction / totalGamesWithFavorites * 100))

In [ ]:
print('Assuming Vegas has a pretty good model')
print('     the underdogs win %6.2f%% of the time' % (sum(newy_all != vegasPredictions) / totalGamesWithFavorites * 100))
print('     the favorites win %6.2f%% of the time' % (sum(newy_all == vegasPredictions) / totalGamesWithFavorites * 100))

In [ ]:
header = ['Team', 'Earnings', 'Total Games', 'Total Investment', 'ROI', 'FavoriteWPercent', 'UnderdogWPercent']
rows = []

def teamPerformance(teamName):
    team = newGamesFrame.loc[(newGamesFrame['HomeTeam'] == teamName) | (newGamesFrame['AwayTeam'] ==  teamName)]
    totalGames = team.shape[0]
    earnings = team['BetFor100Outcome'].sum()
    unequalMatchups = team.loc[team['HomePayout'] != team['AwayPayout']].shape[0]
    favoriteWins = team.loc[team['VegasPrediction'] == team['Result']].shape[0]
#     print("If I bet $100 per game (%d games) on %s, I would make $%6.2f on $%6.2f, %6.2f%% ROI" % (totalGames, teamName, earnings, totalGames * 100, (earnings / totalGames)))
    return [teamName, earnings, totalGames, team['BetFor100TotalInvestment'].sum(), np.float64(earnings / team['BetFor100TotalInvestment'].sum() * 100), np.float64(favoriteWins / unequalMatchups), 1 - (np.float64(favoriteWins / unequalMatchups))]

standings = pd.read_csv(r'/Users/Milton/Developer/Hockey/data/Standings NHL/2018-4-8.csv')
standings.loc[standings['Team'] == 'Arizona Coyotes', 'Team'] = 'Coyotes'

for team in standings[['Team']].values.flatten():
    rows.append(teamPerformance(team))

with open('Bets Team Performance.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(header)
    writer.writerows(rows)
    
bets = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Bets Team Performance.csv')
print('Max ROI: %6.2f%%, Min ROI: %6.2f%%' % (bets['ROI'].max(), bets['ROI'].min()))

In [ ]:
newGamesFrame['ConfidenceDifference'] = abs(newGamesFrame['0_Confidence'] - newGamesFrame['1_Confidence'])
conMax = newGamesFrame['ConfidenceDifference'].max()
conMedian = newGamesFrame['ConfidenceDifference'].median()
conMin = newGamesFrame['ConfidenceDifference'].min()
conMean = newGamesFrame['ConfidenceDifference'].mean()
print('Confidence Difference')
print('     max: %6.2f' % conMax)
print('     min: %6.2f' % conMin)
print('     median: %6.2f' % conMedian)
print('     mean: %6.2f' % conMean)

def calculateAccuracy(threshold):
    confidentFrame = newGamesFrame.loc[newGamesFrame['ConfidenceDifference'] > threshold]
    correctGuesses = confidentFrame.loc[confidentFrame['Result'] == confidentFrame['Prediction']].shape[0]
#     display(confidentFrame[['HomeTeam', 'AwayTeam', 'Prediction', 'VegasPrediction', 'Result', '0_Confidence', '1_Confidence', 'ConfidenceDifference']].sort_values('ConfidenceDifference'))
    print('Threshold: %6.2f, Accuracy: %6.2f%%' % (threshold, (correctGuesses / confidentFrame.shape[0])))
    print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['BetFor100Outcome'].sum(), totalGames * 100))
    
    samePrediction = confidentFrame.loc[confidentFrame['Prediction'] == confidentFrame['VegasPrediction']].shape[0]
    print("My model is able to identify the favourite team %6.2f%% of the time" % (samePrediction / confidentFrame.shape[0] * 100))

for i in [x / 100 for x in range(0, 28)]:
    calculateAccuracy(i)

# calculateAccuracy(0.20)

# print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['BetFor100Outcome'].sum(), totalGames * 100))
# print("If I bet $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['BetWith100Outcome'].sum(), totalGames * 100))
# print("If Vegas bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['VegasBetFor100Outcome'].sum(), totalGames * 100))
# print("If I bet $100 to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['BetEvenOutcome'].sum(), totalGames * 100))

In [ ]:
header = ['Month', 'Accuracy', 'Earnings', 'Total Games', 'Total Investment', 'ROI', 'FavoriteWPercent', 'UnderdogWPercent']
rows = []

def monthPerformance(monthString):
    date = "/%s/" % monthString
    month = newGamesFrame.loc[newGamesFrame['Date'].str.contains(date)]
    totalGames = month.shape[0]
    earnings = month['BetFor100Outcome'].sum()
    unequalMatchups = month.loc[month['HomePayout'] != month['AwayPayout']].shape[0]
    favoriteWins = month.loc[month['VegasPrediction'] == month['Result']].shape[0]
    accuracy = month.loc[month['Prediction'] == month['Result']].shape[0]
#     print("If I bet $100 per game (%d games) on %s, I would make $%6.2f on $%6.2f, %6.2f%% ROI" % (totalGames, teamName, earnings, totalGames * 100, (earnings / totalGames)))
    return [monthString, np.float64(accuracy / totalGames), earnings, totalGames, month['BetFor100TotalInvestment'].sum(), np.float64(earnings / month['BetFor100TotalInvestment'].sum() * 100), np.float64(favoriteWins / unequalMatchups), 1 - (np.float64(favoriteWins / unequalMatchups))]

months = ['11', '12', '01', '02', '03', '04']
for i in months:
    print(i)
    rows.append(monthPerformance(i))

with open('Bets Month Performance.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(header)
    writer.writerows(rows)